In [1]:
from mnist import *

Training dataset size: torch.Size([60000, 28, 28])
Test dataset size: torch.Size([10000, 28, 28])
torch.Size([60000, 1, 28, 28])


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
from torch.utils.cpp_extension import load_inline

def load_cuda(cuda_src, cpp_src, funcs, opt=False, verbose=False):
    return load_inline(cuda_sources=[cuda_src], cpp_sources=[cpp_src], functions=funcs, with_cuda=True,
                       extra_cuda_cflags=["-O3"] if opt else [], verbose=verbose, name="inline_ext")



In [3]:
model.conv1.weight.shape

torch.Size([6, 1, 5, 5])

In [4]:
train_ds.shape

torch.Size([60000, 1, 28, 28])

In [5]:
# weight shape: [o, i, h, w]
# input shape: [b, c, h, w]

In [6]:
cuda_src = open("forward_gpu.cu").read()
cpp_src = """
torch::Tensor forward_conv_gpu(torch::Tensor x, torch::Tensor weight);
"""
funcs = ["forward_conv_gpu"]
ext = load_cuda(cuda_src, cpp_src, funcs)

/home/aki/miniconda3/envs/imitation/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [7]:
#padding
train_ds_pad = torch.nn.functional.pad(train_ds, (2, 2, 2, 2), mode='constant', value=0)

In [8]:
train_ds[:1].shape, model.conv1.weight.shape

(torch.Size([1, 1, 28, 28]), torch.Size([6, 1, 5, 5]))

In [11]:
import time
t0 = time.time()
o = ext.forward_conv_gpu(train_ds_pad[:1000], model.conv1.weight)
print(time.time() - t0)

0.00023889541625976562


In [8]:
# regular conv
#unpad
t0 = time.time()
o2 = model.conv1(train_ds[:1])
print(time.time() - t0)

0.0007305145263671875


In [22]:
torch.allclose(o, o2, atol=1e-5)

True

In [25]:
o = ext.forward_max_pool_cpu(o, 2)

In [26]:
o2 = torch.nn.functional.max_pool2d(o2, 2)